### Single procs code

In [ ]:
import numpy as np
from scipy.sparse import dia_matrix
from math import *
from itertools import combinations
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.signal import find_peaks
import time


start = time.time()

# Required parameters
N = 4           # Insert an integer value for N = no. of spin sites
alpha = 0.0     # the number of order parameter; alpha = 0 is meanfield limit
omegas = np.linspace(0.0,1.0,50)    # Insert a float value for omega
hdc = 0.1       # This is the dc part of the symmetry breaking field
amp = 25.0      # amplitude of the symmetry breaking field
n = 1000         # Number of steps for odeint


#%matplotlib inline
#import matplotlib.pyplot as plt
#from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
#rc('text', usetex=True)
# fs = 15
pheightmin = 0.00010
#fig, axs = plt.subplots(7,2, sharey=True, figsize=(15,15))
fs = 10
# Initial values: ACCORDING To XXZ model
jx, jy, jz = -1.0, 0.0, 0.0 
hx, hy, hz = 0.0, 0.0, -1.0 * amp

# Periodic Boundary Condition
def jmat(N, alpha):
    J = dia_matrix((N, N))
    mid_diag = np.floor(N/2).astype(int)
    for i in np.arange(1,mid_diag+1):
        elem = pow(i, -alpha)
        J.setdiag(elem, k=i)
        J.setdiag(elem, k=-i)
    for i in np.arange(mid_diag+1, N):
        elem = pow(N-i, -alpha)
        J.setdiag(elem, k=i)
        J.setdiag(elem, k=-i)
    return J.toarray()

J = jmat(N,alpha)

def gcorr(G,t,k,a,b,wmat,gmat,v_vec,f_vec):
    left = sum(np.dot(wmat[a,c], gmat[k,c,b]) + np.dot(wmat[b,c], gmat[k,a,c]) for c in np.arange(3))
    mdl1 = v_vec[a] * sum(np.dot(J[0, j], gmat[np.absolute(j-k) + 1, b,0]) for j in np.arange(1,k))
    mdl2 = v_vec[b] * sum(np.dot(J[k, j], gmat[j,0,a]) for j in np.arange(1,k))
    rght = J[0, k] * (np.dot(v_vec[a], f_vec[b]) + np.dot(v_vec[b], f_vec[a]))
    gc   = (left + mdl1 + mdl2 + rght)
    return gc

def g_t(k,i,j,t,gstate,wmat,gmat,v_vec,f_vec):
    d_t = t
    a,b = i,j
    G = odeint(gcorr, gstate, d_t, args=((k,a,b,wmat,gmat,v_vec,f_vec)))
    gstate = G[1]
    return G[1]

def xxz_model(dxyz, tspan, gstate,ts):
    t = ts
    sx, sy, sz = dxyz[0], dxyz[1], dxyz[2]
    
    v_vec = np.array([[0.0], [ 2 * sz], [- 2 * sy]])
    f_vec = np.array([[1 - sx * sx], [- sy * sx], [- sz * sx]])
    hz1 = hz * np.cos(omega * t[1]) + hdc
    wmat = np.array([[0, 2 * hz1, 0], [- 2 * hz1, 0, 2 * (sx + hx)], [0, - 2 * (sx + hx), 0]])

    dsx = 2.0 * hz1 * sy
    dsy = 2.0 * sx * sz + 2.0 * hx * sz - 2.0 * hz1 * sx  + 2.0 * (sum(J[0,j] * g_t(j,0,2,t,gstate,wmat,gmat,v_vec,f_vec) for j in np.arange(1,N)))
    dsz = - 2.0 * sy * sx - 2.0 * hx * sy - 2.0 * sum(J[0,j] * g_t(j,0,1,t,gstate,wmat,gmat,v_vec,f_vec) for j in np.arange(1,N))
    dxyz = [dsx, dsy, dsz]   
    return dxyz

z0 = np.array([1.0, 0.0, 0.0], dtype=np.float64)  # Initial state
t = np.linspace(0.0, 2.0, n)
sx, sy, sz = np.zeros(n), np.zeros(n), np.zeros(n)
sx[0],sy[0],sz[0] = z0[0],z0[1],z0[2]
gstate = 0.0

gmat = np.zeros((N,3,3))
ttt = 0.0              #initialze the time as zero
dq_sx, dq_sz = np.zeros(len(omegas)), np.zeros(len(omegas))
print("run started!")

for omg,omega in enumerate(omegas):
    small_run_s = time.time()
    wmat = np.array([[0, 2 * ( hz * np.cos(omega * ttt) + hdc), 0], \
                          [- 2 * (hz * np.cos(omega * ttt) +hdc), 0, 2 * (sx[0] + hx)],\
                          [0, - 2 * (sx[0] + hx), 0]])
    v_vec = np.array([[0.0], [ 2 * sz[0]], [- 2 * sy[0]]])
    f_vec = np.array([[1 - sx[0] * sx[0]], [- sy[0] * sx[0]], [- sz[0] * sx[0]]])
    for i in range(1,n):
        
        # as the odeint for coupled differential equation runs for single time interval due to its coupled behavior.
        tspan = [t[i-1],t[i]]
        ts = tspan
        dxyz = odeint(xxz_model, z0, tspan, args=((gstate,ts)))
        sx[i], sy[i], sz[i] = dxyz[1][0], dxyz[1][1], dxyz[1][2]
        z0 = dxyz[1] 
        #filename = 'bbgky_t_sx_sz_N4_w25p0_alpha' + str(alph) +'.txt'
    
    spectrum_sx = np.fft.fftshift(np.fft.fft(sx - np.average(sx)))
    spectrum_sz = np.fft.fftshift(np.fft.fft(sz - np.average(sz)))
    peaks1, _ = find_peaks(np.abs(spectrum_sx), height=pheightmin)
    peaks2, _ = find_peaks(np.abs(spectrum_sz), height=pheightmin)
    #print(spectrum_sx)
    dq_sx[omg] = np.amax(np.abs(spectrum_sx[peaks1]))
    dq_sz[omg] = np.amax(np.abs(spectrum_sz[peaks2]))
    small_run_f = time.time()
    print("omega=",omega,"time",small_run_f-small_run_s,"sec")
print("total time taken",time.time()-start,"sec")    
plt.plot(omegas, dq_sx,label='fft_sx')
plt.plot(omegas, dq_sz,label="fft_sz")
plt.legend()
plt.show()